In [0]:
# !pip install --upgrade tables
# !pip install eli5
# !pip install xgboost

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [5]:
ls

data/  day2_visualization.ipynb  day3_simple_model.ipynb  LICENSE  README.md


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mea
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

Import danych

In [8]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

# Feature Engineering

In [0]:
SUFFIX_CAT = "__cat"

for feat in df.columns:
  
  if isinstance( df[feat][0], list ): continue
  
  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values


In [10]:
cat_feats = [ x for x in df.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree

In [14]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost


In [16]:
xgb_params = {
    "max_depth": 5,
    "n_estimators": 50,
    "learning_rate": 0.1, 
    "seed":0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[07:50:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:51:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:51:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

## Waznosc cech

In [17]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[07:53:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [0]:
feats = ["param_napęd__cat","param_rok-produkcji__cat","param_stan__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_moc__cat","param_marka-pojazdu__cat","feature_kamera-cofania__cat","param_typ__cat","param_pojemność-skokowa__cat","seller_name__cat","feature_wspomaganie-kierownicy__cat","param_model-pojazdu__cat","param_wersja__cat","param_kod-silnika__cat","feature_system-start-stop__cat","feature_asystent-pasa-ruchu__cat","feature_czujniki-parkowania-przednie__cat","feature_łopatki-zmiany-biegów__cat","feature_regulowane-zawieszenie__cat"]

In [19]:
len(feats)

20

Testowanie modelu na zmniejszonej liczbie cech

In [20]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[08:02:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

Analiza zawartości cech

Wartość numeryczna (rok) była utracona przez "factorize".
Aby przywrócić "fajną ciągłość" danych mapujemy 'none' na -1 i trenujemy model raz jeszcze tylko z ficzerem 'param_rok-produkcji' zamiast 'param_rok-produkcji__cat'

In [27]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', -1, '1975', '1973', '1953',
       '1985', '1984', '1986', '1981', '1979', '1960', '1983', '1978',
       '1964', '1980', '1972', '1969', '1956', '1966', '1977', '1962',
       '1965', '1971', '1963', '1961', '1952', '1949', '1976', '1937',
       '1968', '1958', '1955', '1933', '1929', '1957', '1944', '1954',
       '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else x)

In [26]:
feats = ["param_napęd__cat","param_rok-produkcji","param_stan__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_moc__cat","param_marka-pojazdu__cat","feature_kamera-cofania__cat","param_typ__cat","param_pojemność-skokowa__cat","seller_name__cat","feature_wspomaganie-kierownicy__cat","param_model-pojazdu__cat","param_wersja__cat","param_kod-silnika__cat","feature_system-start-stop__cat","feature_asystent-pasa-ruchu__cat","feature_czujniki-parkowania-przednie__cat","feature_łopatki-zmiany-biegów__cat","feature_regulowane-zawieszenie__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[09:02:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:02:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:02:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

To samo robimy dla parametru "moc"

In [0]:
df["param_moc"] = df["param_moc"].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [37]:
feats = ["param_napęd__cat","param_rok-produkcji","param_stan__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_moc","param_marka-pojazdu__cat","feature_kamera-cofania__cat","param_typ__cat","param_pojemność-skokowa__cat","seller_name__cat","feature_wspomaganie-kierownicy__cat","param_model-pojazdu__cat","param_wersja__cat","param_kod-silnika__cat","feature_system-start-stop__cat","feature_asystent-pasa-ruchu__cat","feature_czujniki-parkowania-przednie__cat","feature_łopatki-zmiany-biegów__cat","feature_regulowane-zawieszenie__cat"]
 
run_model(xgb.XGBRegressor(**xgb_params), feats)

[09:12:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:12:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:12:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

To samo dla pojemności skokowej

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))

In [51]:
df['param_pojemność-skokowa'].unique()

array([   898,   1560,   3000, ...,   5992,   1966, 142280])

In [48]:
feats = ["param_napęd__cat","param_rok-produkcji","param_stan__cat","param_skrzynia-biegów__cat","param_faktura-vat__cat","param_moc","param_marka-pojazdu__cat","feature_kamera-cofania__cat","param_typ__cat","param_pojemność-skokowa","seller_name__cat","feature_wspomaganie-kierownicy__cat","param_model-pojazdu__cat","param_wersja__cat","param_kod-silnika__cat","feature_system-start-stop__cat","feature_asystent-pasa-ruchu__cat","feature_czujniki-parkowania-przednie__cat","feature_łopatki-zmiany-biegów__cat","feature_regulowane-zawieszenie__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[09:17:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:17:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:17:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)